In [ ]:
#데이터확인
import numpy as np
import pandas as pd
import glob
from tqdm import tqdm
import datetime


#데이터 시각화
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#기타
import warnings
warnings.filterwarnings("ignore")

# 1. 데이터 확인

## Train.csv 확인

In [ ]:
train = pd.read_csv("../input/predict-volcanic-eruptions-ingv-oe/train.csv")
train.head()

In [ ]:
train.info()

In [ ]:
train_csvs = glob.glob("../input/predict-volcanic-eruptions-ingv-oe/train/*")
test_csvs = glob.glob("../input/predict-volcanic-eruptions-ingv-oe/test/*")

## Train segment_id 확인

In [ ]:
train_csvs[0]  #위에 glob으로 파일패스를 만들었기떄문에 요렇게쓰면 경로지정
sequence = pd.read_csv(train_csvs[0]) #위의 경로에있는 파일 불러오기
sequence

In [ ]:
sequence.describe()

## Sensor 값 확인하기

In [ ]:
#plt를 사용하여 간단하게 시각화
def show_sensors(df):
    f, axes = plt.subplots(10, 1)
    f.set_size_inches((16, 10)) 
    f.tight_layout()
    plt.subplots_adjust(bottom=-0.4)
    
    # Sensor #1 ~ #10
    for i in range(1,11):
        axes[i-1].plot(df[f'sensor_{i}'].values)
        axes[i-1].set_title('Sensor_'+str(i))
        axes[i-1].set_xlabel('time')
        
show_sensors(sequence)

In [ ]:
#널값을 채워준다
show_sensors(sequence.fillna(0))

# 2. 데이터 분석 - Basic EDA

## Train.csv
- `time_to_eruption`을 h:m:s(시:분:초)로 변환

In [ ]:
train.head()

- 센서 값은 10분 간 `60000`개가 측정됨
- 10분은 600(10 * 60초)초 이므로, 센서 값은 0.01초 간격으로 샘플링됨(단위가 10e-2 초 - centisecond 임)
- `datetime.timedelta` 적용할 때, 위 특성을 적용

In [ ]:
#위의 0.0초단위를 시:분:초로 변환
train['time_to_eruption'].apply(lambda x:datetime.timedelta(seconds = x/100))

In [ ]:
#밑의 시각화를 위해 새로운 컬럼 생성
train['hhmmss'] = train['time_to_eruption'].apply(lambda x:datetime.timedelta(seconds = x/100))
train

In [ ]:
#평균적으로 몇일뒤에 화산이 발생하는지 확인
plt.figure(figsize=(16,8))
plt.hist(train['hhmmss'] / pd.Timedelta(days=1))
plt.xlabel('Time between eruptions (days)')
plt.ylabel('# of eruptions');

In [ ]:
(train['hhmmss'] / pd.Timedelta(hours=1)).hist()

In [ ]:
#px로도 시각화 가능 좀더 유동적인 것을 가능
fig = px.histogram(
    train, 
    x="time_to_eruption",
    width=800,
    height=600,
    nbins=10,
    title='Time to eruption distribution'
)

fig.show()

In [ ]:
fig = px.line(
    train, 
    y="time_to_eruption",
    width=800,
    height=500,
    title='Time to eruption for all volcanos'
)

fig.show()

## Train & Test 셋 분석

## Train 셋

In [ ]:
#중복이 있는지 확인,센서값이 모두 10개인지 확인 널값 채우기
sensors = set()
observations = set()
nan_columns = list()
missed_groups = list()
for_df = list()

for item in train_csvs:           #train_csvs폴더에서 파일들을 하나하나 가져옴
    name = int(item.split('.')[-2].split('/')[-1])   #파일이름에서.전까지 즉 확장자 뺴고 이름만 가져옴
    at_least_one_missed = 0
    frag = pd.read_csv(item)     #판다스로 데이터를 읽는다
    missed_group = list()
    missed_percents = list()
    for col in frag.columns:
        missed_percents.append(frag[col].isnull().sum() / len(frag))
        if pd.isnull(frag[col]).all() == True:     #각각의 컬럼에대해 널값이 있는지 확
            at_least_one_missed = 1     #널값이 있다면 1을 저
            nan_columns.append(col)     #어펜드로 위의 non_coloumns에 넣는다
            missed_group.append(col)
    if len(missed_group) > 0:
        missed_groups.append(missed_group)
    sensors.add(len(frag.columns))
    observations.add(len(frag))
    for_df.append([name, at_least_one_missed] + missed_percents)

In [ ]:
print('고유한(Unique) 센서 수: ', sensors)
print('고유한(Unique) 행(row) 수: ', observations)

In [ ]:
#넑값 체
print('센서 값이 비어있는(missed) 센서 수:', len(nan_columns))

absent_sensors = dict()

for item in nan_columns:
    if item in absent_sensors:
        absent_sensors[item] += 1
    else:
        absent_sensors[item] = 0

In [ ]:
#널값의 시각화
absent_df = pd.DataFrame(absent_sensors.items(), columns=['Sensor', 'Missed sensors'])

fig = px.bar(
    absent_df, 
    x="Sensor",
    y='Missed sensors',
    width=800,
    height=500,
    title='Number of missed sensors in training dataset'
)

fig.show()

## Test 셋

In [ ]:
sensors = set()
observations = set()
nan_columns = list()
missed_groups = list()
for_test_df = list()

for item in test_csvs:
    name = int(item.split('.')[-2].split('/')[-1])
    at_least_one_missed = 0
    frag = pd.read_csv(item)
    missed_group = list()
    missed_percents = list()
    for col in frag.columns:
        missed_percents.append(frag[col].isnull().sum() / len(frag))
        if pd.isnull(frag[col]).all() == True:
            at_least_one_missed = 1
            nan_columns.append(col)
            missed_group.append(col)
    if len(missed_group) > 0:
        missed_groups.append(missed_group)
    sensors.add(len(frag.columns))
    observations.add(len(frag))
    for_test_df.append([name, at_least_one_missed] + missed_percents)

In [ ]:
print('고유한(Unique) 센서 수: ', sensors)
print('고유한(Unique) 행(row) 수: ', observations)

In [ ]:
print('값이 비어있는(missed) 센서 수:', len(nan_columns))

absent_sensors = dict()

for item in nan_columns:
    if item in absent_sensors:
        absent_sensors[item] += 1
    else:
        absent_sensors[item] = 0

In [ ]:
absent_df = pd.DataFrame(absent_sensors.items(), columns=['Sensor', 'Missed sensors'])

fig = px.bar(
    absent_df, 
    x="Sensor",
    y='Missed sensors',
    width=800,
    height=500,
    title='Number of missed sensors in test dataset'
)

fig.show()